In [1]:
import os
import math
import torch
import gpytorch
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from tqdm import tqdm
from gpytorch.kernels import ScaleKernel, MaternKernel, RQKernel, RBFKernel, SpectralMixtureKernel

In [2]:
# Wrap some useful functions here

def train(model: gpytorch.models, likelihood: gpytorch.likelihoods, training_iters: int, 
        train_x: torch.tensor, train_y: torch.tensor, lr:float=0.1, patience_iters:int=10, patience:float=1e-3, verbose:bool=False):
    """ Train a model with Adam optimizer and marginal log likelihood.
        :param model: 
            a valid gpytorch GP model (which returns gpytorch.distributions)
        :param likelihood:
            specifies the mapping from latent function values f to observations y
        :param training_iters:
            times of training iterations
        :param train_x:
            inputs for training, with shape (n_samples, n_features)
        :param train_y:
            (scalar) outputs of train_x, with shape (n_samples, )
        :param lr:
            learning rate of the optimizer, default 0.1
        :param patience_iters:
            controls the waiting iters of the loss varaition
        :param patience:
            controls the variation rate of loss for early stop
        :param verbose:
            controls whether printing the training info
    """
    # use Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # use marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    counter = 0
    memoryloss = None
    if verbose:
        with tqdm(total=training_iters) as t:
            for i in range(training_iters):
                t.set_description("Epoch %d, Early-Stop Counter %d" % (i, counter))
                optimizer.zero_grad()
                output = model(train_x)
                loss = -mll(output, train_y)
                loss.backward()
                optimizer.step()
                t.set_postfix(loss=loss.item())
                t.update(1)

                if i > 0:
                    if abs(loss.item() - memoryloss) / abs(memoryloss) < patience:
                        counter += 1
                    else:
                        counter = 0

                memoryloss = loss.item()

                if counter >= patience_iters:
                    print("Early Stop Triggered!")
                    break
    else:
        for i in range(training_iters):
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
            if i > 0:
                if abs(loss.item() - memoryloss) / abs(memoryloss) < patience:
                    counter += 1
                else:
                    counter = 0
            memoryloss = loss.item()
            if counter >= patience_iters:
                break

def predict(model: gpytorch.models, likelihood: gpytorch.likelihoods, 
        test_x: torch.tensor, is_observed:bool=True):
    """ Use a trained model to make predictions.
        :param model: 
            a valid gpytorch GP model (which returns gpytorch.distributions)
        :param likelihood:
            specifies the mapping from latent function values f to observations y
        :param test_x:
            inputs to predict, with shape (n_samples, n_features)
        :param is_observed:
            whether the outputs should be observed or latent outputs
        :return:
            outputs at the input points
    """
    model.eval()
    likelihood.eval()

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        if is_observed:
            return likelihood(model(test_x))
        else:
            return model(test_x)

def plot(predictions: gpytorch.distributions, train_x: torch.tensor,
        train_y: torch.tensor, test_x: torch.tensor):
    """ Plot the predicted results (2-D only).
        :param predictions:
            gpytorch.distributions, usually MultivariateNormal
        :param train_x:
            the input points of training, with shape (n_samples, )
        :param train_y:
            the output points of training, with shape (n_samples, )
        :param test_x:
            the input points to predict, with shape (n_samples, )
    """
    with torch.no_grad():
        f, ax = plt.subplots(1, 1, figsize=(4,3))
        # confidence bounds
        lower, upper = predictions.confidence_region()
        # training data is plotted as black stars
        ax.plot(train_x.cpu().numpy(), train_y.cpu().numpy(), "k*")
        # predicted mean is plotted as blue line
        ax.plot(test_x.cpu().numpy(), predictions.mean.cpu().numpy(), "b")
        # shade the confidence region
        ax.fill_between(test_x.cpu().numpy(), lower.cpu().numpy(), upper.cpu().numpy(), alpha=0.5)
        # legend & x/ylim
        # ax.set_ylim([-3,3])
        ax.legend(['Observed Data', 'Mean', 'Confidence'])

In [3]:
# Customized Kernel for Trend Forecasting
class NSKernel(gpytorch.kernels.Kernel):
    is_stationary = False
    has_lengthscale = True

    def __init__(self, num_mixtures:int, ard_num_dims:int, nu:float, **kwargs):
        super().__init__(**kwargs)
        self.spmk = SpectralMixtureKernel(num_mixtures, ard_num_dims)
        self.mk = MaternKernel(nu)
        self.k1 = ScaleKernel(self.spmk * self.mk)
        self.rbf = RBFKernel()
        self.k2 = ScaleKernel(self.rbf)
        self.ns_mapper = torch.nn.Sequential(
            torch.nn.Linear(ard_num_dims, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 4),
        )
    
    def forward(self, x1, x2, last_dim_is_batch=False):
        y1 = self.k1(x1,x2)
        y2 = self.k2(self.ns_mapper(x1), self.ns_mapper(x2))
        return self.lengthscale * (y1+y2)

In [4]:
# Kernel for Seasonal Component Forecasting
# NOTE: may need improvement
class PeriodicKernel(gpytorch.kernels.Kernel):
    # the sinc kernel is stationary
    is_stationary = True

    # We will register the parameter when initializing the kernel
    def __init__(self, length_prior=None, length_constraint=None, **kwargs):
        super().__init__(**kwargs)

        # register the raw parameter
        self.register_parameter(
            name='raw_length', parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 1, 1))
        )

        # set the parameter constraint to be positive, when nothing is specified
        if length_constraint is None:
            length_constraint = gpytorch.constraints.Positive()

        # register the constraint
        self.register_constraint("raw_length", length_constraint)

        # set the parameter prior, see
        # https://docs.gpytorch.ai/en/latest/module.html#gpytorch.Module.register_prior
        if length_prior is not None:
            self.register_prior(
                "length_prior",
                length_prior,
                lambda m: m.length,
                lambda m, v : m._set_length(v),
            )

    # now set up the 'actual' paramter
    @property
    def length(self):
        # when accessing the parameter, apply the constraint transform
        return self.raw_length_constraint.transform(self.raw_length)

    @length.setter
    def length(self, value):
        return self._set_length(value)

    def _set_length(self, value):
        if not torch.is_tensor(value):
            value = torch.as_tensor(value).to(self.raw_length)
        # when setting the paramater, transform the actual value to a raw one by applying the inverse transform
        self.initialize(raw_length=self.raw_length_constraint.inverse_transform(value))

    # this is the kernel function
    def forward(self, x1, x2, **params):
        # apply lengthscale
        x1_ = x1.div(self.length)
        x2_ = x2.div(self.length)
        # calculate the distance between inputs
        diff = self.covar_dist(x1_, x2_, **params)
        # prevent divide by 0 errors
        diff.where(diff == 0, torch.as_tensor(1e-20).cuda())
        # return sinc(diff) = sin(diff) / diff
        return torch.sin(diff).div(diff)

In [5]:
# Load Data
# Forecasting Horizons: hourly-48 daily-14 weekly-13 monthly-18 quarterly-8 yearly-6
dataset_path = "./Dataset/"
train_data_path = dataset_path+"Train/"
test_data_path = dataset_path+"Test/"
train_datasets = os.listdir(train_data_path)
test_datasets = os.listdir(test_data_path)

print(train_datasets)
print(test_datasets)

['Daily-train.csv', 'Hourly-train.csv', 'Monthly-train.csv', 'Quarterly-train.csv', 'Weekly-train.csv', 'Yearly-train.csv']
['Daily-test.csv', 'Hourly-test.csv', 'Monthly-test.csv', 'Quarterly-test.csv', 'Weekly-test.csv', 'Yearly-test.csv']


In [6]:
train_data = pd.read_csv(train_data_path + train_datasets[-1],index_col=0)
test_data = pd.read_csv(test_data_path + test_datasets[-1],index_col=0)
print(train_data.shape)
print(test_data.shape)

(23000, 835)
(23000, 6)


In [7]:
# use torch.utils.data.Dataset as data container
from torch.utils.data import Dataset, DataLoader
from typing import Tuple

class GPTimeserieDataset(Dataset):
    """ Accept the concatenation of training and testing data 
        in pd.DataFrame (n_series, series_len)
        return the available torch.Tensors (TrainData, TestData)

        :data_train: the pd.DataFrame (n_series, series_len)
        :data_test: forecasting pointrs (n_series, forecast_len)
        :window_size: training/testing point for learning, should be forecasting points + 1
        :date_freq: frequency between each points of used timeseries 
        :max_len: if specified, only use series_len smaller than this parameter
    """
    def __init__(self, data_train:pd.DataFrame, data_test:pd.DataFrame,
                date_freq:str, window_size:int=None, max_len:int=None):
        # pre-process the data
        if max_len is not None:
            valid_idx = []
            for idx,col in data_train.T.items():
                if col.dropna().shape[0] <= max_len:
                    valid_idx.append(idx)
            data_train = data_train.loc[valid_idx, :]
            data_test = data_test.loc[valid_idx, :]
        else:
            data_train = data_train
            data_test = data_test

        self.data_train = data_train
        self.data_test = data_test
        self.date_freq = date_freq
        if window_size is None:
            self.window_size = data_test.shape[-1] + 1
        else:
            self.window_size = window_size
        self.forecast_pts = data_test.shape[-1]
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"

    def _proc_log(self, x: torch.Tensor) -> torch.Tensor:
        x_ = tuple(x[i:i+self.window_size] \
            for i in range(x.shape[0] - self.window_size+1))
        x_ = tuple(torch.log(i).reshape(-1,1) for i in x_)
        return torch.cat(x_, dim=1)

    def _proc(self, x: torch.Tensor) -> torch.Tensor:
        x_ = tuple(x[i:i+self.window_size].reshape(-1,1) \
            for i in range(x.shape[0] - self.window_size+1))
        return torch.cat(x_, dim=1)
        
    def __len__(self):
        assert self.data_train.shape[0] == self.data_test.shape[0]
        return self.data_train.shape[0]

    def __getitem__(self, index) -> \
        Tuple[Tuple[torch.Tensor, torch.Tensor], Tuple[torch.Tensor, torch.Tensor], \
        Tuple[torch.Tensor, torch.Tensor], Tuple[torch.Tensor, torch.Tensor], torch.Tensor]:
        """ Return ((trend_train_x, trend_train_y), (trend_test_x, trend_test_y), 
            (seasonal_train_x, seasonal_train_y), (seasonal_test_x, seasonal_test_y), original)
        """
        train = self.data_train.iloc[index, :].dropna()
        test = self.data_test.iloc[index, :].dropna()
        d = pd.concat((train, test)).values
        t = pd.date_range(start="2000-01-01", periods=d.shape[0], freq=self.date_freq)
        ts = pd.DataFrame(d).set_index(t)
        rslt = seasonal_decompose(ts, model="m", extrapolate_trend="freq")
        trends = torch.Tensor(rslt.trend.values).clamp_(1e-6).to(self.device)
        seasons = torch.Tensor(rslt.seasonal.values).clamp_(1e-6).to(self.device)
        original = torch.Tensor(d).to(self.device)
        trends = self._proc_log(trends).T
        seasons = self._proc(seasons).T
        trends_train_x = trends[:-self.forecast_pts, :-1]
        trends_train_y = trends[:-self.forecast_pts, -1]
        trends_test_x = trends[-self.forecast_pts:, :-1]
        trends_test_y = trends[-self.forecast_pts:, -1]
        seasons_train_x = seasons[:-self.forecast_pts, :-1]
        seasons_train_y = seasons[:-self.forecast_pts, -1]
        seasons_test_x = seasons[-self.forecast_pts:, :-1]
        seasons_test_y = seasons[-self.forecast_pts:, -1]
        return (
            (trends_train_x, trends_train_y), (trends_test_x, trends_test_y),
            (seasons_train_x, seasons_train_y), (seasons_test_x, seasons_test_y), original
        )

In [8]:
# benchmarks drawn from github.com/Mcompetitions/M4-methods
def smape(a, b):
    """
    Calculates sMAPE
    :param a: actual values
    :param b: predicted values
    :return: sMAPE
    """
    a = np.reshape(a, (-1,))
    b = np.reshape(b, (-1,))
    return np.mean(2.0 * np.abs(a - b) / (np.abs(a) + np.abs(b))).item()


def mase(insample, y_test, y_hat_test, freq):
    """
    Calculates MAsE
    :param insample: insample data
    :param y_test: out of sample target values
    :param y_hat_test: predicted values
    :param freq: data frequency
    :return:
    """
    # this func only works with 1d arrays
    insample = insample.squeeze()
    y_test = y_test.squeeze()
    y_hat_test = y_hat_test.squeeze()

    y_hat_naive = []
    for i in range(freq, len(insample)):
        y_hat_naive.append(insample[(i - freq)])

    masep = np.mean(abs(insample[freq:] - y_hat_naive))

    return np.mean(abs(y_test - y_hat_test)) / masep

# inverse transformation of trend & seasonal components
def ts_trans(trend: torch.Tensor, seasonal: torch.Tensor) -> np.array:
    return (torch.exp(trend) * seasonal).cpu().numpy()

In [9]:
# Define the GP
class trendGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(trendGP, self).__init__(train_x, train_y, likelihood)
        self.mean = gpytorch.means.ConstantMean()
        self.cov = NSKernel(4,6,1.5)
    def forward(self, x):
        mean = self.mean(x)
        cov = self.cov(x)
        return gpytorch.distributions.MultivariateNormal(mean, cov)

class seasonalGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(seasonalGP, self).__init__(train_x, train_y, likelihood)
        self.mean = gpytorch.means.ConstantMean()
        self.cov = PeriodicKernel()
    def forward(self, x):
        mean = self.mean(x)
        cov = self.cov(x)
        return gpytorch.distributions.MultivariateNormal(mean, cov)

In [ ]:
# Go training
from linear_operator.utils.errors import NotPSDError
smape_list = []
mase_list = []

dataloader = DataLoader(GPTimeserieDataset(train_data, test_data, "1d"), shuffle=True)
counter = 0

for ((t_train_x, t_train_y), (t_test_x, t_test_y), \
    (s_train_x, s_train_y), (s_test_x, s_test_y), origin) in tqdm(dataloader):
    
    counter += 1
    if counter >= 1000:
        break

    try:
        t_likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()
        t_model = trendGP(t_train_x, t_train_y, t_likelihood).cuda()
        s_likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()
        s_model = seasonalGP(s_train_x, s_train_y, s_likelihood).cuda()
        train(t_model, t_likelihood, 1000, t_train_x, t_train_y, lr=0.04, patience=1e-2)
        train(s_model, s_likelihood, 1000, s_train_x, s_train_y, lr=0.05)
        t_pred_y = predict(t_model, t_likelihood, t_test_x)
        s_pred_y = predict(s_model, s_likelihood, s_test_x)
        origin = origin.squeeze()
        smape_list.append(smape(ts_trans(t_test_y, s_test_y), ts_trans(t_pred_y.mean, s_pred_y.mean)))
        mase_list.append(mase(ts_trans(t_train_y, s_train_y), ts_trans(t_test_y, s_test_y),\
            ts_trans(t_pred_y.mean, s_pred_y.mean), freq=1))
    except NotPSDError:
        continue

In [ ]:
smape_mean = np.mean(np.array(smape_list))
mase_mean = np.mean(np.array(mase_list))
print(smape_mean, mase_mean)

In [38]:
# Following part is copied from previous work.
# define load data func
def load_data_from_csv(filepath: str) -> np.ndarray:
    csv_file = pd.read_csv(filepath)
    school_num = csv_file.shape[0]
    year_num = csv_file.shape[1] - 2    # except first and last column
    data_dim = len(csv_file.iloc[0, 1].strip().split())
    school_names = tuple(csv_file["SchoolName"])
    data = np.zeros((school_num, year_num, data_dim))
    for i, _ in enumerate(school_names):
        data_row = list(csv_file.iloc[i, 1:year_num+1])
        data_row = [_.strip().split() for _ in data_row]
        data_row = [float(_) for l in data_row for _ in l]
        data_row = np.array(data_row).reshape((year_num, -1))
        data[i, :, :] = data_row
    return data, school_names

data, school_names = load_data_from_csv("./Dataset/UE/all-data.csv")
school_n, year_n, data_dim = data.shape
idx2sch = {k:v for k,v in enumerate(school_names)}

selected_data_idx = np.array([0, 2, 7, 11, 15, 19])
selected_data = data[:, :, selected_data_idx]
selected_data = selected_data.transpose(0, 2, 1)
print(selected_data.shape)

more_data_path = "./Dataset/UE/more-school-all.npy"
more_data = np.load(more_data_path)
selected_data = np.concatenate((selected_data, more_data), axis=0)
print(selected_data.shape)

reshaped = selected_data.reshape(-1, 10)
train_data = pd.DataFrame(reshaped[:, :-1])
test_data = pd.DataFrame(reshaped[:, -1])

train_data[train_data <= 0] = 1e-6
test_data[test_data <= 0] = 1e-6

print(train_data.shape, test_data.shape)

(20, 6, 10)
(50, 6, 10)
(300, 9) (300, 1)


In [39]:
# Define the GP
class trendGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(trendGP, self).__init__(train_x, train_y, likelihood)
        self.mean = gpytorch.means.ConstantMean()
        self.cov = NSKernel(4,1,1.5)
    def forward(self, x):
        mean = self.mean(x)
        cov = self.cov(x)
        return gpytorch.distributions.MultivariateNormal(mean, cov)

class seasonalGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(seasonalGP, self).__init__(train_x, train_y, likelihood)
        self.mean = gpytorch.means.ConstantMean()
        self.cov = PeriodicKernel()
    def forward(self, x):
        mean = self.mean(x)
        cov = self.cov(x)
        return gpytorch.distributions.MultivariateNormal(mean, cov)

In [40]:
# Go training
from linear_operator.utils.errors import NotPSDError
smape_list = []
mase_list = []

dataloader = DataLoader(GPTimeserieDataset(train_data, test_data, "1y"), shuffle=True)
counter = 0

for ((t_train_x, t_train_y), (t_test_x, t_test_y), \
    (s_train_x, s_train_y), (s_test_x, s_test_y), origin) in tqdm(dataloader):
    
    counter += 1
    if counter >= 1000:
        break

    try:
        t_likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()
        t_model = trendGP(t_train_x, t_train_y, t_likelihood).cuda()
        s_likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()
        s_model = seasonalGP(s_train_x, s_train_y, s_likelihood).cuda()
        train(t_model, t_likelihood, 1000, t_train_x, t_train_y, lr=0.04, patience=1e-2)
        train(s_model, s_likelihood, 1000, s_train_x, s_train_y, lr=0.05)
        t_pred_y = predict(t_model, t_likelihood, t_test_x)
        s_pred_y = predict(s_model, s_likelihood, s_test_x)
        origin = origin.squeeze()
        smape_list.append(smape(ts_trans(t_test_y, s_test_y), ts_trans(t_pred_y.mean, s_pred_y.mean)))
        mase_list.append(mase(ts_trans(t_train_y, s_train_y), ts_trans(t_test_y, s_test_y),\
            ts_trans(t_pred_y.mean, s_pred_y.mean), freq=1))
    except NotPSDError:
        continue

100%|██████████| 300/300 [16:37<00:00,  3.32s/it]


In [ ]:
smape_mean = np.mean(np.array(smape_list))
mase_mean = np.mean(np.array(mase_list))
print(smape_mean, mase_mean)